In [1]:
"""
Script de scraping Yahoo Finance - Version Production (Securisee).

Fonctionnalites :
- Reprise automatique la ou le script s'est arrete.
- Gestion manuelle des CAPTCHAs Google sans plantage.
- Sauvegarde atomique pour eviter toute corruption du CSV.
- Aucun emoji, logs propres.
"""

import time
import random
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# --- CONFIGURATION ---
CSV_FILENAME = "cac40_transcripts_full_2025.csv"
TARGET_YEAR = "2025"
QUARTERS = ["Q1", "Q2", "Q3", "Q4"]

# Liste complete des entreprises
CAC40_COMPANIES = [
    "LVMH", "TotalEnergies", "Sanofi", "Airbus", "Schneider Electric",
    "Air Liquide", "L'Oreal", "BNP Paribas", "AXA", "Vinci",
    "Safran", "Hermes", "Kering", "EssilorLuxottica", "Capgemini",
    "Stellantis", "Credit Agricole", "Orange", "Pernod Ricard", "Engie",
    "Michelin", "Legrand", "Publicis", "Teleperformance", "Veolia",
    "Saint-Gobain", "Societe Generale", "Danone", "Carrefour", "Renault",
    "Thales", "ArcelorMittal", "STMicroelectronics", "Eurofins Scientific",
    "Vivendi", "Unibail-Rodamco-Westfield", "Edenred", "Bouygues"
]

def setup_driver():
    """Configure le navigateur Chrome."""
    options = Options()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("start-maximized")
    
    # Masquer l'automatisation pour reduire la detection
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("--disable-blink-features=AutomationControlled")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def load_existing_data():
    """
    Charge les donnees existantes pour permettre la reprise.
    Gere les fichiers vides ou corrompus.
    """
    if os.path.exists(CSV_FILENAME):
        try:
            df = pd.read_csv(CSV_FILENAME)
            if df.empty:
                return [], set()
            
            # On cree un identifiant unique (Entreprise + Trimestre)
            # pour savoir ce qui est deja fait.
            existing_records = df.to_dict('records')
            existing_keys = set(f"{row['company']}_{row['quarter']}" for row in existing_records)
            
            print(f"[INFO] Reprise : {len(existing_records)} transcripts deja recuperes.")
            return existing_records, existing_keys
            
        except pd.errors.EmptyDataError:
            print("[INFO] Fichier CSV existant mais vide. On recommence.")
            return [], set()
        except Exception as e:
            print(f"[ATTENTION] Erreur lecture CSV ({e}). On recommence a zero par securite.")
            return [], set()
            
    return [], set()

def save_data_safely(data):
    """
    Sauvegarde atomique pour eviter la corruption de donnees.
    Ecrit dans un fichier .tmp puis le renomme.
    """
    temp_filename = CSV_FILENAME + ".tmp"
    try:
        df = pd.DataFrame(data)
        df.to_csv(temp_filename, index=False)
        
        # Remplacement atomique du fichier principal
        os.replace(temp_filename, CSV_FILENAME)
    except Exception as e:
        print(f"[ERREUR CRITIQUE] Impossible de sauvegarder : {e}")

def check_for_captcha(driver):
    """
    Detecte la presence d'un CAPTCHA Google et met le script en pause
    pour permettre une resolution manuelle par l'utilisateur.
    """
    try:
        page_text = driver.page_source.lower()
        # Detection basique des mots cles de blocage
        if "recaptcha" in page_text or "unusual traffic" in page_text or "trafic inhabituel" in page_text:
            print("\n" + "!"*60)
            print("[ALERTE] CAPTCHA GOOGLE DETECTE")
            print("Action requise :")
            print("1. Ouvrez la fenetre Chrome du script.")
            print("2. Resolvez le CAPTCHA manuellement.")
            print("3. Attendez l'affichage des resultats de recherche.")
            print("4. Revenez ici et appuyez sur ENTREE.")
            print("!"*60)
            
            input("Appuyez sur ENTREE une fois le CAPTCHA resolu pour continuer...")
            
            print("[INFO] Reprise du script...")
            time.sleep(2)
            return True
    except Exception:
        pass
    return False

def google_search_quarter_safe(driver, company, quarter):
    """Effectue la recherche Google avec gestion d'erreurs."""
    query = f'site:finance.yahoo.com "{company}" "{quarter} {TARGET_YEAR}" "Earnings Call Transcript"'
    print(f"[RECHERCHE] {company} {quarter}...")

    try:
        driver.get("https://www.google.com")
        time.sleep(random.uniform(1.5, 2.5))

        # Tentative de clic sur les cookies Google
        try:
            buttons = driver.find_elements(By.TAG_NAME, "button")
            for btn in buttons:
                if "tout accepter" in btn.text.lower() or "accept all" in btn.text.lower():
                    driver.execute_script("arguments[0].click();", btn)
                    time.sleep(1)
                    break
        except Exception:
            pass

        # Saisie de la recherche
        try:
            search_box = driver.find_element(By.NAME, "q")
            search_box.clear()
            search_box.send_keys(query)
            search_box.send_keys(Keys.RETURN)
        except Exception:
            # Si l'element n'est pas trouve, c'est peut-etre un Captcha immediat
            check_for_captcha(driver)
            return []

        # Attente aleatoire "humaine"
        time.sleep(random.uniform(3, 5))
        
        # Verification post-recherche
        check_for_captcha(driver)

        # Parsing des resultats
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        links_found = []
        
        for h3 in soup.find_all('h3'):
            link_tag = h3.find_parent('a')
            if link_tag and 'href' in link_tag.attrs:
                url = link_tag['href']
                title = h3.get_text()

                # Filtrage strict
                if "finance.yahoo.com" in url and "transcript" in title.lower():
                    # Verification que le titre contient bien le trimestre demande
                    # pour eviter les faux positifs (ex: Google donne Q1 quand on cherche Q2)
                    if quarter.lower() in title.lower() or quarter in title:
                        if "presentation" not in title.lower():
                            links_found.append({
                                "company": company,
                                "quarter": quarter,
                                "year": TARGET_YEAR,
                                "title": title,
                                "url": url
                            })
        
        return links_found[:1]

    except Exception as e:
        print(f"[ERREUR] Recherche Google : {e}")
        return []

def extract_content(driver, url):
    """Extrait le texte de la page Yahoo/Quartr."""
    try:
        driver.get(url)
        time.sleep(random.uniform(3, 6))
        
        # Gestion cookies Yahoo
        if "consent" in driver.current_url:
            try:
                driver.find_element(By.NAME, "agree").click()
                time.sleep(3)
            except Exception:
                pass
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Priorite a la structure Quartr (vue dans vos captures)
        content = soup.select_one('div[class*="transcriptContainer"]')
        
        # Fallback structure classique
        if not content:
            content = soup.find('div', class_='caas-body')

        if content:
            # Nettoyage HTML
            for tag in content(["script", "style", "button", "iframe", "div.caas-instream"]):
                tag.decompose()
            text = content.get_text(separator="\n", strip=True)
            return text
            
    except Exception as e:
        print(f"[ERREUR] Extraction contenu : {e}")
    return None

def main():
    print("--- Demarrage du script de scraping (Mode Securise) ---")
    
    # 1. Chargement intelligent des donnees
    all_data, done_set = load_existing_data()
    
    driver = setup_driver()
    
    try:
        for company in CAC40_COMPANIES:
            for quarter in QUARTERS:
                # Creation de la cle unique pour verification
                task_key = f"{company}_{quarter}"
                
                # Si deja fait, on passe au suivant sans rien faire
                if task_key in done_set:
                    continue

                # Execution de la recherche
                results = google_search_quarter_safe(driver, company, quarter)
                
                if not results:
                    print(f"[ECHEC] Pas de transcript trouve pour {company} {quarter}")
                    time.sleep(random.uniform(2, 4))
                    continue

                item = results[0]
                text = extract_content(driver, item['url'])
                
                if text and len(text) > 1000:
                    item['content'] = text
                    
                    # Mise a jour des listes en memoire
                    all_data.append(item)
                    done_set.add(task_key)
                    
                    print(f"[SUCCES] {company} {quarter} sauvegarde ({len(text)} carac.)")
                    
                    # SAUVEGARDE SUR DISQUE SECURISEE
                    save_data_safely(all_data)
                else:
                    print(f"[AVERTISSEMENT] Contenu vide ou trop court pour {item['title']}")

                # Pause longue pour menager Google
                time.sleep(random.uniform(5, 10))
                
            print("-" * 30)

    except KeyboardInterrupt:
        print("\n[STOP] Arret manuel par l'utilisateur. Les donnees sont en securite.")
    except Exception as e:
        print(f"\n[ERREUR FATALE] {e}")
    finally:
        driver.quit()
        print(f"[FIN] Traitement termine. Fichier : {CSV_FILENAME}")

if __name__ == "__main__":
    main()

--- Demarrage du script de scraping (Mode Securise) ---
[INFO] Reprise : 54 transcripts deja recuperes.
[RECHERCHE] LVMH Q4...
[ECHEC] Pas de transcript trouve pour LVMH Q4
------------------------------
[RECHERCHE] TotalEnergies Q4...
[ECHEC] Pas de transcript trouve pour TotalEnergies Q4
------------------------------
------------------------------
------------------------------
[RECHERCHE] Schneider Electric Q4...
[ECHEC] Pas de transcript trouve pour Schneider Electric Q4
------------------------------
[RECHERCHE] Air Liquide Q4...

[STOP] Arret manuel par l'utilisateur. Les donnees sont en securite.
[FIN] Traitement termine. Fichier : cac40_transcripts_full_2025.csv
